In [1]:
#import package
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data as ds
from model import RNN,LSTM,GRU
import torch.nn.functional as F
import ast
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np


In [2]:
trainset=pd.read_csv('ESGOverallData Trainset.csv')
trainset.dropna(inplace=True)
testset = pd.read_csv('ESGOverallData Testset.csv')
testset.dropna(inplace=True)
predict = pd.read_csv('ESGOverallData Predictset.csv')
predict.dropna(inplace=True)

In [3]:
display(trainset)

,CompanyTicker,Year,CSV.Length,ESGData,BloombergOverall
0,ON,2016,1131,"[[0.0055219400674104, 0.0078070629388093, 0.00...",4.16
1,FFIV,2020,844,"[[0.004199112765491, 0.0238876864314079, 0.021...",2.70
2,NVDA,2016,451,"[[0.0117304930463433, 0.015321378596127, 0.010...",5.68
3,CSCO,2016,692,"[[0.0129457851871848, 0.0162092838436365, 0.01...",4.53
4,SYNA,2018,697,"[[0.0032364279031753, 0.0065713287331163, 0.00...",1.65
...,...,...,...,...,...
266,KLAC,2016,1156,"[[0.0019834351260215, 0.0017482317052781, 0.00...",2.33
267,AMD,2017,1360,"[[0.0024161378387361, 0.001941418624483, 0.003...",6.09
268,AGS,2018,259,"[[0.0055219400674104, 0.0078070629388093, 0.00...",1.23
269,SYNA,2015,380,"[[0.0129457851871848, 0.0162092838436365, 0.01...",1.84


In [4]:
#data = [
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.2], [0.1, 0.2, 0.3, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.1]],   # report1 representation
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.3]],
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.2], [0.1, 0.2, 0.3, 0.5]],   # report2 representation
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.3], [0.1, 0.2, 0.3, 0.2], [0.1, 0.2, 0.3, 0.5]]    # report2 representation
#]

def trim_data(data, max_length=3):
    trimmed_data = []
    for sample in data:
        if len(sample) > max_length:
            # 找出最里面第四个维度的值
            fourth_values = [item[3] for item in sample]
            # 找出要保留的前max_length个最大值的索引
            indices_to_keep = sorted(range(len(fourth_values)), key=lambda i: fourth_values[i], reverse=False)[:max_length]
            # 按原顺序保留这些索引对应的元素
            trimmed_report = [sample[i] for i in sorted(indices_to_keep)]
            trimmed_data.append(trimmed_report)
        else:
            trimmed_data.append(sample)
    return trimmed_data

#trimmed_data = trim_data(data)
#print(trimmed_data)


In [5]:
#regression dataset
class MyDataset1(ds.Dataset):
    def __init__(self, X, Y):
        self.samples = X
        self.labels = Y

    def __getitem__(self, index):
        sample = self.samples[index]
        label = [self.labels[index]]
        #print(label)
        sample=torch.Tensor(sample).float()
        label= torch.Tensor(label).float()

        return sample, label

    def __len__(self):
        return len(self.samples)

In [6]:
def generate_data(df):
    x_train=[]
    y_train=[]
    for index,row in df.iterrows():
        x_train.append(ast.literal_eval(row["ESGData"]))
        y_train.append(float(row["BloombergOverall"]))
    return x_train,y_train


In [7]:
def pad_time_series(data, target_length=None, padding_value=0.0):
    # Determine the target length if not provided
    final_result=[]
    if target_length is None:
        target_length = max(len(sample) for sample in data)

    for sample in data:
        if len(sample) < target_length:
            for _ in range(target_length - len(sample)):
                sample.append([padding_value for _ in range(4)])
            final_result.append(sample)
        
        elif len(sample) > target_length:
            fourth_values = [item[3] for item in sample]
            indices_to_keep = sorted(range(len(fourth_values)), key=lambda i: fourth_values[i], reverse=False)[:target_length]
            trimmed_report = [sample[i] for i in sorted(indices_to_keep)]
            final_result.append(trimmed_report)
        
        else:
            final_result.append(sample)

    return final_result
            

# Example usage:
#data = [
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.2], [0.1, 0.2, 0.3, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.1]],   # report1 representation
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.3]],
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.3],[0.1, 0.2, 0.3, 0.3]],
#   [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.2], [0.1, 0.2, 0.3, 0.5]],   # report2 representation
#    [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.3, 0.3], [0.1, 0.2, 0.3, 0.2], [0.1, 0.2, 0.3, 0.5]]    # report2 representation
#]


In [8]:

def mean_absolute_error(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))
def r_squared(y_true, y_pred,y_mean):
    ss_res = torch.sum((y_true - y_pred) ** 2)
    ss_tot = torch.sum((y_true - y_mean) ** 2)
    return 1 - (ss_res / ss_tot)
def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100
def mean_squared_error(y_true, y_pred):
    return torch.mean((y_true - y_pred) ** 2)
def information_coefficient(y_true, y_pred):
    # 计算IC值
    numerator = torch.corrcoef(y_true, y_pred)[0, 1]  # 相关系数
    denominator = torch.std(y_true) * torch.std(y_pred)  # 标准差的乘积
    return numerator / denominator

In [9]:
#training loop
def loop(model,train_loader,test_loader,optimizer,criterion,device,E=20):
    loss_all=[] # List to store the loss for each batch in the epoch
    print(f'Device is using:{device}')
    for epoch in range(E): 
        loss_epoch=[]
        #print(epoch)
        for samples, labels in train_loader:
            samples=samples.to(device)
            labels=labels.to(device)
            #print(samples.shape)
            #print(labels.shape)
            model.train()               # Set the flag to training mode
            optimizer.zero_grad()       # Clear the gradients of all optimized tensors
            outputs = model(samples)    # Forward pass
            loss = criterion(outputs.squeeze(), labels) # Compute the loss
            loss.backward()     # Backward pass
            optimizer.step()    # Update the parameters
            loss_epoch.append(float(loss.cpu()))    # Store the loss value for the batch
            #print(outputs)
        #print(f'MSE{sum(loss_epoch)/len(loss_epoch)}') #训练集平均loss
        loss_all.append(sum(loss_epoch)/len(loss_epoch))    # Store the average loss for the epoch

    model.eval() # Set the model to evaluation mode
    result=[]   # List to store the predictions on the test dataset
    TrueValue = []
    mae_loss = 0
    r2_score = 0
    mape_loss = 0
    mse_loss = 0  # Initialize MSE loss
    ic = 0

    total_labels = torch.tensor([]).to(device) # Initialize total_labels with an empty tensor
    for samples, true in test_loader:
        samples=samples.to(device)  # Move samples to the specified device
        outputs = model(samples) # 预测值  Forward pass to get predictions
        predictions = float(outputs.cpu()) # loat(outputs.cpu())
        result.append(predictions)
        TrueValue.append(float(true.cpu()))

        # Update total_labels with the new batch of labels
        total_labels = torch.cat((total_labels, labels))
        # Calculate the current y_mean
        y_mean = torch.mean(total_labels)

        # Compute MAE
        mae_loss += mean_absolute_error(true, predictions)        
        # Compute R^2
        r2_score += r_squared(true, predictions,y_mean)        
        # Compute MAPE
        mape_loss += mean_absolute_percentage_error(true, predictions)        
        # Compute MSE
        mse_loss += mean_squared_error(true, predictions)

    # Average the metrics over the test dataset
    num_samples = len(test_loader)
    mae_loss /= num_samples
    r2_score /= num_samples
    mape_loss /= num_samples
    mse_loss /= num_samples
    ic, _ = pearsonr(result, TrueValue)
    rank_ic, _ = spearmanr(result, TrueValue)

    AverageMetrics = [mae_loss,r2_score,mape_loss,mse_loss,ic]
        
    return result,loss_all,model,AverageMetrics

nn.MSELoss(), the criterion variable can be used with other loss functions depending on the type of problem you are trying to solve. 

nn.CrossEntropyLoss() for classification tasks, which computes the cross-entropy loss between the predicted and true class labels.

nn.BCEWithLogitsLoss() for binary classification tasks, which combines the sigmoid function with the binary cross-entropy loss.

nn.NLLLoss() for classification tasks, which computes the negative log likelihood loss.

nn.HingeLoss() for classification tasks, especially for support vector machines, which computes the hinge loss between the predicted and true values.

nn.L1Loss() for regression tasks, which computes the mean absolute error loss.
nn.SmoothL1Loss() for regression tasks, which computes a smooth version of the mean absolute error loss.

# RNN

In [10]:
#parameters
batch_size=180 #128
learning_rate=0.001      #0.000001
epoch=150
hidden_num=50
output_dim=1 #if None put ''
device = torch.device("cpu")

In [11]:
x,y=generate_data(trainset)
x = pad_time_series(x, 800, padding_value=0.25)
train_dataset = MyDataset1(x,y)
x,y=generate_data(testset)
x = pad_time_series(x, 800, padding_value=0.25)
test_dataset = MyDataset1(x,y)

train_loader=ds.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader=ds.DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)
criterion = nn.MSELoss()
display(test_loader)

In [12]:
features_dim=np.array(x).shape[2]
modelRNN=RNN.RNN_Model(features_dim,hidden_num,output_dim).to(device)
optimizerRNN= optim.Adam(modelRNN.parameters(), lr=learning_rate)
resultRNN,lossRNN,modelRNN,AverageMetrics=loop(modelRNN,train_loader,test_loader,optimizerRNN,criterion,device,E=epoch)
print(f'RNN predict {resultRNN}')
print(f'loss RNN {lossRNN}')
print(f'AverageMetrics RNN {AverageMetrics}')

/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device is using:cpu


/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([180, 1])) that is different to the input size (torch.Size([180])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([91, 1])) that is different to the input size (torch.Size([91])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RNN predict [2.872462272644043, 2.872462272644043, 2.872462272644043, 2.8720884323120117, 2.870527744293213, 2.872462272644043, 2.872096538543701, 2.8724617958068848, 2.871939182281494, 2.8720126152038574, 2.872462272644043, 2.872462272644043, 2.8720977306365967, 2.872462272644043, 2.8717355728149414, 2.8726511001586914, 2.872462272644043, 2.8721137046813965, 2.872462272644043, 2.872462272644043, 2.872044086456299, 2.872121572494507, 2.872061014175415, 2.872462272644043, 2.872462272644043, 2.872462272644043, 2.87211012840271, 2.872248411178589, 2.872462272644043, 2.872462272644043, 2.872462272644043, 2.8721182346343994, 2.8724122047424316, 2.872462272644043, 2.872220516204834, 2.872462272644043, 2.8719472885131836, 2.8721132278442383, 2.8720736503601074, 2.872462272644043, 2.872073173522949, 2.872462272644043, 2.8724639415740967, 2.872462272644043, 2.872462272644043, 2.872462272644043, 2.872462272644043, 2.8720712661743164, 2.872436761856079, 2.872462272644043, 2.872462272644043, 2.872

# LSTM

In [13]:
#parameters
batch_size=180 #128
learning_rate=0.01      #0.000001
epoch=100
hidden_num=100
output_dim=1 #if None put ''
device = torch.device("cpu")

In [14]:
x,y=generate_data(trainset)
x = pad_time_series(x, 800, padding_value=0.25)
train_dataset = MyDataset1(x,y)
x,y=generate_data(testset)
x = pad_time_series(x, 800, padding_value=0.25)
test_dataset = MyDataset1(x,y)

train_loader=ds.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader=ds.DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)
criterion = nn.MSELoss()

In [15]:
features_dim=np.array(x).shape[2]
modelLSTM=LSTM.LSTM_Model(features_dim,hidden_num,output_dim).to(device)
optimizerLSTM= optim.Adam(modelLSTM.parameters(), lr=learning_rate)
resultLSTM,lossLSTM,modelLSTM,AverageMetrics=loop(modelLSTM,train_loader,test_loader,optimizerLSTM,criterion,device,E=epoch)
print(f'LSTM predict {resultLSTM}')
print(f'loss LSTM {lossLSTM}')
print(f'AverageMetrics LSTM {AverageMetrics}')

Device is using:cpu
LSTM predict [2.8284823894500732, 2.8288726806640625, 2.8285467624664307, 2.8284945487976074, 2.8288726806640625, 2.8288726806640625, 2.828462600708008, 2.8284857273101807, 2.8285152912139893, 2.8288726806640625, 2.828463554382324, 2.8288726806640625, 2.8284683227539062, 2.828460216522217, 2.8288726806640625, 2.8288726806640625, 2.828460931777954, 2.8288726806640625, 2.8288726806640625, 2.8288726806640625, 2.8288726806640625, 2.8288726806640625, 2.8288726806640625, 2.82846736907959, 2.8284878730773926, 2.8284640312194824, 2.8288726806640625, 2.8284904956817627, 2.8288726806640625, 2.8288726806640625, 2.828474283218384, 2.8288726806640625, 2.8288726806640625, 2.828486442565918, 2.8288726806640625, 2.8285040855407715, 2.8288726806640625, 2.8284571170806885, 2.8288726806640625, 2.828458309173584, 2.8288726806640625, 2.8288726806640625, 2.8284575939178467, 2.8284757137298584, 2.8288726806640625, 2.828451156616211, 2.828451156616211, 2.828461170196533, 2.8288726806640625

# GRU

In [16]:
#parameters
batch_size=150 #128
learning_rate=0.01      #0.000001
epoch=100
hidden_num=100
output_dim=1 #if None put ''
device = torch.device("cpu")

In [17]:
x,y=generate_data(trainset)
x = pad_time_series(x, 800, padding_value=0.25)
train_dataset = MyDataset1(x,y)
x,y=generate_data(testset)
x = pad_time_series(x, 800, padding_value=0.25)
test_dataset = MyDataset1(x,y)

train_loader=ds.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader=ds.DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)
criterion = nn.MSELoss()

In [18]:
features_dim=np.array(x).shape[2]
modelGRU=GRU.GRU_Model(features_dim,hidden_num,output_dim).to(device)
optimizerGRU= optim.Adam(modelGRU.parameters(), lr=learning_rate)
resultGRU,lossGRU,modelGRU,AverageMetrics=loop(modelGRU,train_loader,test_loader,optimizerGRU,criterion,device,E=epoch)
print(f'GRU predict {resultGRU}')
print(f'loss GRU {lossGRU}')
print(f'AverageMetrics GRU {AverageMetrics}')

Device is using:cpu


/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([150, 1])) that is different to the input size (torch.Size([150])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/qjy/anaconda3/envs/NLP/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([121, 1])) that is different to the input size (torch.Size([121])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


GRU predict [0.08248762786388397, 0.08248761296272278, 0.05618912726640701, 0.0517645962536335, 0.08248762786388397, 0.08242166042327881, 0.05140719562768936, 0.08248763531446457, 0.08248762786388397, 0.08248762786388397, 0.08248762786388397, 0.08248763531446457, 0.053810618817806244, 0.05644446983933449, 0.08248763531446457, 0.054227281361818314, 0.054361071437597275, 0.08248763531446457, 0.05727456510066986, 0.08248763531446457, 0.05582272261381149, 0.053490884602069855, 0.08248762786388397, 0.08248763531446457, 0.05223724991083145, 0.08248763531446457, 0.08248762786388397, 0.08248794078826904, 0.05604394152760506, 0.05985166132450104, 0.08248762786388397, 0.05428798869252205, 0.054909247905015945, 0.08248762041330338, 0.056560732424259186, 0.055657707154750824, 0.05400438606739044, 0.05661492794752121, 0.08248762786388397, 0.08248762041330338, 0.05774550512433052, 0.05051557719707489, 0.08248763531446457, 0.08248762786388397, 0.05572814121842384, 0.08248762786388397, 0.0824876353144